# Cleaning Datasets for Gentrification Analysis

Each dataset below is from the City of New York's open data. I will be using each of these to contribute to an analysis of how different areas of NYC have changed over the last ~10 years. 

In [2]:
# import statments
import pandas as pd
import numpy as np

#### PLUTO (Primary Tax-Use Land Output) Dataset

This dataset contains important information for each building, including: location, building classification codes (what each building is used for), property assesment values, zoning information, and more. My other datasets contain information like evictions, sale prices, building permits, buyers/sellers; PLUTO contains information about the type of buildings in which those events happened.

In [35]:
# Cleaning PLUTO

# Load PLUTO
pluto = pd.read_csv('Primary_Land_Use_Tax_Lot_Output_(PLUTO)_20251112.csv', low_memory=False)

print(f"Original PLUTO shape: {pluto.shape}")

# Keep columns relevant for gentrification analysis
columns_to_keep = [
    'BBL',                  # Borough-Block-Lot ID (unique property identifier)
    'borough',              # Borough code (MN, BX, BK, QN, SI)
    'Tax block',            # Tax block number
    'Tax lot',              # Tax lot number
    'address',              # Street address
    'postcode',             # ZIP code
    'community board',      # Community board district
    'census tract 2010',    # Census tract (for demographic data)
    'bldgclass',            # Building classification code (A=single family, B=2-family, C/D=apartments)
    'landuse',              # Land use category code
    'ownertype',            # Owner type (P=Private, C=City, M=Mixed, O=Other)
    'ownername',            # Property owner name (helps identify institutional owners)
    'numbldgs',             # Number of buildings on the lot
    'numfloors',            # Number of floors in building
    'unitsres',             # Number of residential units (KEY for gentrification)
    'unitstotal',           # Total number of units (residential + commercial)
    'lotarea',              # Lot area in square feet
    'bldgarea',             # Total building area in square feet
    'resarea',              # Residential area in square feet
    'comarea',              # Commercial area in square feet
    'yearbuilt',            # Year building was built
    'yearalter1',           # Year of first major alteration
    'yearalter2',           # Year of second major alteration
    'histdist',             # Historic district name (if applicable)
    'landmark',             # Landmark designation (if applicable)
    'assessland',           # Assessed land value
    'assesstot',            # Total assessed value (land + building)
    'exempttot',            # Total tax exemptions
    'latitude',             # Latitude coordinate
    'longitude'             # Longitude coordinate
]

# Select only the columns we want
pluto_clean = pluto[columns_to_keep].copy()

# Mark residential properties (building classes A, B, C, D)
pluto_clean['is_residential'] = pluto_clean['bldgclass'].astype(str).str[0].isin(['A', 'B', 'C', 'D'])

# Remove invalid BBLs
pluto_clean = pluto_clean[pluto_clean['BBL'].notna()]
pluto_clean['BBL'] = pluto_clean['BBL'].astype(str).str.replace('.0', '', regex=False)
pluto_clean = pluto_clean[pluto_clean['BBL'].str.len() == 10]

# Convert numeric columns
numeric_cols = ['numfloors', 'unitsres', 'unitstotal', 'lotarea', 'bldgarea', 
                'resarea', 'comarea', 'assessland', 'assesstot', 'exempttot', 'yearbuilt']
for col in numeric_cols:
    pluto_clean[col] = pd.to_numeric(pluto_clean[col], errors='coerce')

print(f"\nCleaned PLUTO shape: {pluto_clean.shape}")
print(f"Residential properties: {pluto_clean['is_residential'].sum():,}")

# BBL example meaning: 1001230045 = Manhattan (1), Block 00123, Lot 0045
print(f"BBL sample: {pluto_clean['BBL'].head().tolist()}") 

# Save
pluto_clean.to_csv('pluto_cleaned.csv', index=False)
print("\n✓ PLUTO cleaned and saved")

Original PLUTO shape: (857736, 101)

Cleaned PLUTO shape: (857736, 31)
Residential properties: 716,254
BBL sample: ['4064210038', '4051750020', '4051730111', '4051740016', '4064130046']

✓ PLUTO cleaned and saved


#### Evictions Dataset

This dataset's purpose is pretty self-explanatory. Places with increasing rent and new groups of people moving in means that currrent residents will be displaced.

In [36]:
# Cleaning Evictions

# Load evictions
evictions = pd.read_csv('Evictions_20251112.csv', low_memory=False)

print(f"Original evictions shape: {evictions.shape}")

# Keep relevant columns
columns_to_keep = [
    'Court Index Number',           # Unique identifier for the case
    'Docket Number ',               # Another case identifier (note the space!)
    'Eviction Address',             # Street address
    'Eviction Apartment Number',    # Apartment number (for counting evictions per building)
    'Executed Date',                # When eviction actually happened (KEY)
    'Residential/Commercial',       # Property type
    'BOROUGH',                      # Borough
    'Eviction Postcode',            # ZIP code
    'BBL',                          # Property ID (to link with PLUTO, ACRIS, etc.)
    'Latitude',                     # For mapping
    'Longitude',                    # For mapping
    'Community Board',              # Neighborhood district
    'Census Tract',                 # For demographic analysis
    'NTA'                           # Neighborhood Tabulation Area
]

evictions_clean = evictions[columns_to_keep].copy()

# Clean column names (remove trailing spaces)
evictions_clean.columns = evictions_clean.columns.str.strip()

# Convert date to datetime
evictions_clean['Executed Date'] = pd.to_datetime(evictions_clean['Executed Date'], errors='coerce')

# Filter to only executed evictions (remove rows where Executed Date is missing)
evictions_clean = evictions_clean[evictions_clean['Executed Date'].notna()].copy()

print(f"\nAfter filtering to executed evictions: {evictions_clean.shape}")

# Filter to only RESIDENTIAL evictions (for gentrification focus)
evictions_clean['Residential/Commercial'] = evictions_clean['Residential/Commercial'].str.strip().str.upper()
evictions_clean = evictions_clean[evictions_clean['Residential/Commercial'] == 'RESIDENTIAL'].copy()

print(f"After filtering to residential only: {evictions_clean.shape}")

# Create time-based columns for analysis
evictions_clean['eviction_year'] = evictions_clean['Executed Date'].dt.year
evictions_clean['eviction_month'] = evictions_clean['Executed Date'].dt.month
evictions_clean['eviction_year_month'] = evictions_clean['Executed Date'].dt.to_period('M').astype(str)

# Clean borough names
evictions_clean['BOROUGH'] = evictions_clean['BOROUGH'].str.strip().str.upper()

# Clean BBL (ensure it's string with no decimals)
evictions_clean['BBL'] = evictions_clean['BBL'].astype(str).str.replace('.0', '', regex=False)
evictions_clean['BBL'] = evictions_clean['BBL'].str.strip()

# Remove rows with missing BBL (can't link to other datasets without it)
evictions_clean = evictions_clean[evictions_clean['BBL'].notna()]
evictions_clean = evictions_clean[evictions_clean['BBL'] != '']
evictions_clean = evictions_clean[evictions_clean['BBL'] != 'nan']

# Clean addresses
evictions_clean['Eviction Address'] = evictions_clean['Eviction Address'].str.strip().str.upper()

print(f"\nFinal cleaned evictions: {evictions_clean.shape}")

# Summary statistics
print("\n=== Evictions by Year ===")
print(evictions_clean['eviction_year'].value_counts().sort_index())

print("\n=== Evictions by Borough ===")
print(evictions_clean['BOROUGH'].value_counts())

print("\n=== Date Range ===")
print(f"Earliest: {evictions_clean['Executed Date'].min()}")
print(f"Latest: {evictions_clean['Executed Date'].max()}")

# Check for multiple evictions at same address
print("\n=== Buildings with Multiple Evictions ===")
evictions_per_bbl = evictions_clean.groupby('BBL').size().reset_index(name='eviction_count')
print(f"Buildings with 5+ evictions: {(evictions_per_bbl['eviction_count'] >= 5).sum()}")
print(f"Buildings with 10+ evictions: {(evictions_per_bbl['eviction_count'] >= 10).sum()}")

# Save cleaned data
evictions_clean.to_csv('evictions_cleaned.csv', index=False)
print("\n✓ Evictions cleaned and saved!")

# Optional: Save the eviction counts per building for quick analysis
evictions_per_bbl.to_csv('evictions_per_bbl.csv', index=False)
print("✓ Eviction counts per building saved to evictions_per_bbl.csv")

Original evictions shape: (118771, 20)

After filtering to executed evictions: (118771, 14)
After filtering to residential only: (108122, 14)

Final cleaned evictions: (100917, 17)

=== Evictions by Year ===
eviction_year
2017    19219
2018    18455
2019    15836
2020     2853
2021      147
2022     3821
2023    11438
2024    14657
2025    14491
Name: count, dtype: int64

=== Evictions by Borough ===
BOROUGH
BRONX            34499
BROOKLYN         28157
QUEENS           19216
MANHATTAN        15653
STATEN ISLAND     3392
Name: count, dtype: int64

=== Date Range ===
Earliest: 2017-01-03 00:00:00
Latest: 2025-11-10 00:00:00

=== Buildings with Multiple Evictions ===
Buildings with 5+ evictions: 4447
Buildings with 10+ evictions: 1302

✓ Evictions cleaned and saved!
✓ Eviction counts per building saved to evictions_per_bbl.csv


#### ACRIS Datasets

##### NOTE: THE CODE FROM HERE DOWN IS IRRELAVANT TO FINAL PRESENTATION, I DID NOT END UP USING THESE DATASETS

There are three ACRIS files. Property Master contains transaction details. Property Legals contains property information. Property Parties details buyer/seller details.

In [ ]:
# Clean ACRIS Master

# Load files
print("Loading ACRIS Master...")
master = pd.read_csv('ACRIS_-_Real_Property_Master_20251112.csv', low_memory=False)

print("Loading Document Control Codes...")
doc_codes = pd.read_csv('ACRIS_-_Document_Control_Codes_20251112.csv')

print(f"\nOriginal Master records: {len(master):,}")

# Check what document types we have
print("\n=== Document Types in Master ===")
print(master['DOC. TYPE'].value_counts().head(20))

# Find all DEED-related document types from control codes
deed_types = doc_codes[
    doc_codes['CLASS CODE DESCRIPTION'] == 'DEEDS AND OTHER CONVEYANCES'
]['DOC. TYPE'].unique()

print(f"\n=== DEED Document Types (from control codes) ===")
print(deed_types)

# Filter Master to only DEED transactions
master_deeds = master[master['DOC. TYPE'].isin(deed_types)].copy()
print(f"\nDeed transactions: {len(master_deeds):,}")

# Convert dates
master_deeds['DOC. DATE'] = pd.to_datetime(master_deeds['DOC. DATE'], errors='coerce')
master_deeds['RECORDED / FILED'] = pd.to_datetime(master_deeds['RECORDED / FILED'], errors='coerce')

# Filter to 2015 onwards
master_deeds = master_deeds[master_deeds['DOC. DATE'] >= '2015-01-01']
print(f"Deeds from 2015+: {len(master_deeds):,}")

# Clean document amount
master_deeds['DOC. AMOUNT'] = pd.to_numeric(master_deeds['DOC. AMOUNT'], errors='coerce')

# Filter out zero/low amounts (gifts, nominal transfers)
master_deeds = master_deeds[master_deeds['DOC. AMOUNT'] > 1000]
print(f"After filtering low amounts: {len(master_deeds):,}")

# Keep essential columns
master_clean = master_deeds[[
    'DOCUMENT ID',      # To link to legals & parties
    'BOROUGH',          # Borough code
    'DOC. TYPE',        # Document type
    'DOC. DATE',        # Transaction date
    'DOC. AMOUNT',      # Sale price
    'RECORDED / FILED', # Recording date
    '% TRANSFERRED'     # Partial vs full transfer
]].copy()

# Create year column for analysis
master_clean['sale_year'] = master_clean['DOC. DATE'].dt.year

print("\n=== Sales by Year ===")
print(master_clean['sale_year'].value_counts().sort_index())

print("\n=== Sales by Borough ===")
borough_map = {1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'}
master_clean['borough_name'] = master_clean['BOROUGH'].map(borough_map)
print(master_clean['borough_name'].value_counts())

# Save cleaned master
master_clean.to_csv('acris_master_cleaned.csv', index=False)
print(f"\n✓ Saved {len(master_clean):,} deed records to acris_master_cleaned.csv")

# Extract document IDs for filtering parties
doc_ids = master_clean['DOCUMENT ID'].unique()
print(f"\n✓ Found {len(doc_ids):,} unique document IDs")

# Save doc IDs for filtering parties
pd.DataFrame({'document_id': doc_ids}).to_csv('document_ids_for_parties.csv', index=False)
print("✓ Saved document IDs to document_ids_for_parties.csv")

Loading ACRIS Master...
Loading Document Control Codes...

Original Master records: 16,834,719

=== Document Types in Master ===
DOC. TYPE
MTGE        4185390
DEED        3602605
SAT         2599314
ASST        2189753
PAT         1045344
AGMT         912048
RPTT&RET     385121
RPTT         211052
AL&R         194600
REL          169384
TL&R         145830
SAGE         132797
PREL         116896
RTXL         108331
DEED, TS      90671
M&CON         71914
SUBM          59625
SMIS          57799
DTL           56542
CERT          53998
Name: count, dtype: int64

=== DEED Document Types (from control codes) ===
['ASTU' 'CNTR' 'CODP' 'DCTO' 'DEED' 'EASE' 'IDED' 'LEAS' 'LTPA' 'SUBL'
 'VAC' 'CDEC' 'DEEDO' 'DEVR' 'DECM' 'MLEA' 'MCON' 'ACON' 'CORRD' 'CONDEED'
 'REIT' 'NAPP' 'DEED COR' 'DEED, LE' 'CORR, LE' 'DEED, TS' 'AIRRIGHT'
 'SI CORR' 'DEEDP' 'TORREN' 'DEED, RC' 'SCDEC' 'TODD' 'RTOD']

Deed transactions: 3,868,681
Deeds from 2015+: 719,339
After filtering low amounts: 415,193

=== Sales by 

In [ ]:
# Clean ACRIS Legals

print("\n" + "="*60)
print("Loading ACRIS Legals...")
legals = pd.read_csv('ACRIS_-_Real_Property_Legals_20251112.csv', low_memory=False)

print(f"Original Legals records: {len(legals):,}")

# Filter to only our document IDs
legals_filtered = legals[legals['DOCUMENT ID'].isin(doc_ids)].copy()
print(f"Filtered Legals records: {len(legals_filtered):,}")

# Create BBL from Borough, Block, Lot
legals_filtered['BBL'] = (
    legals_filtered['BOROUGH'].astype(str).str.zfill(1) +
    legals_filtered['BLOCK'].astype(str).str.zfill(5) +
    legals_filtered['LOT'].astype(str).str.zfill(4)
)

# Keep essential columns
legals_clean = legals_filtered[[
    'DOCUMENT ID',
    'BOROUGH',
    'BLOCK',
    'LOT',
    'BBL',
    'STREET NUMBER',
    'STREET NAME',
    'UNIT'
]].copy()

# Remove duplicates (some documents have multiple lots)
# For now, keep first BBL per document
legals_clean = legals_clean.drop_duplicates(subset=['DOCUMENT ID'], keep='first')

legals_clean.to_csv('acris_legals_cleaned.csv', index=False)
print(f"✓ Saved {len(legals_clean):,} legal records to acris_legals_cleaned.csv")


Loading ACRIS Legals...
Original Legals records: 22,401,682
Filtered Legals records: 453,001
✓ Saved 414,018 legal records to acris_legals_cleaned.csv


In [ ]:
# Clean ACRIS Parties

print("\n" + "="*60)
print("Loading ACRIS Parties...")
parties = pd.read_csv('ACRIS_-_Real_Property_Parties_20251112.csv', low_memory=False)

print(f"Original Parties records: {len(parties):,}")

# Filter to only our document IDs
parties_filtered = parties[parties['DOCUMENT ID'].isin(doc_ids)].copy()
print(f"Parties for our documents: {len(parties_filtered):,}")

# Check party types
print("\n=== Party Types ===")
print(parties_filtered['PARTY TYPE'].value_counts())

# Based on doc codes: PARTY2 = GRANTEE/BUYER for deeds
# Filter to buyers only (adjust party type code if needed)
buyers = parties_filtered[parties_filtered['PARTY TYPE'] == 2].copy()
print(f"\nBuyers only: {len(buyers):,}")

# Clean names
buyers['NAME'] = buyers['NAME'].str.strip().str.upper()

# Identify institutional buyers
institutional_keywords = [
    'LLC', 'L.L.C', 'L L C',
    'CORP', 'CORPORATION', 
    'INC', 'INCORPORATED',
    'LTD', 'LIMITED',
    'LP', 'L.P', 'LLP',
    'TRUST',
    'FUND',
    'CAPITAL',
    'HOLDINGS',
    'PROPERTIES',
    'REALTY',
    'PARTNERS',
    'VENTURES',
    'INVESTMENTS',
    'EQUITIES',
    'GROUP',
    'MANAGEMENT',
    'ASSETS'
]

def is_institutional(name):
    if pd.isna(name):
        return False
    name_upper = str(name).upper()
    return any(keyword in name_upper for keyword in institutional_keywords)

buyers['is_institutional'] = buyers['NAME'].apply(is_institutional)

print(f"\n=== Buyer Analysis ===")
print(f"Total buyers: {len(buyers):,}")
print(f"Institutional buyers: {buyers['is_institutional'].sum():,} ({buyers['is_institutional'].mean()*100:.1f}%)")
print(f"Individual buyers: {(~buyers['is_institutional']).sum():,}")

# Keep essential columns
buyers_clean = buyers[[
    'DOCUMENT ID',
    'PARTY TYPE',
    'NAME',
    'is_institutional',
    'ADDRESS 1',
    'ADDRESS 2',
    'CITY',
    'STATE',
    'ZIP',
    'COUNTRY'
]].copy()

buyers_clean.to_csv('acris_buyers_cleaned.csv', index=False)
print(f"\n✓ Saved {len(buyers_clean):,} buyer records to acris_buyers_cleaned.csv")


Loading ACRIS Parties...
Original Parties records: 45,868,796


NameError: name 'doc_ids' is not defined

##### Merge All Acris Data

In [ ]:
print("\n" + "="*60)
print("Merging all ACRIS data...")

# Merge master + legals (to get BBL)
acris = master_clean.merge(
    legals_clean[['DOCUMENT ID', 'BBL', 'STREET NUMBER', 'STREET NAME']],
    on='DOCUMENT ID',
    how='left'
)

# Merge with buyers (to get buyer info)
acris = acris.merge(
    buyers_clean[['DOCUMENT ID', 'NAME', 'is_institutional']],
    on='DOCUMENT ID',
    how='left'
)

print(f"\nFinal merged dataset: {len(acris):,} records")
print(f"Records with BBL: {acris['BBL'].notna().sum():,}")
print(f"Records with buyer info: {acris['NAME'].notna().sum():,}")

# Rename columns for clarity
acris = acris.rename(columns={
    'DOC. DATE': 'sale_date',
    'DOC. AMOUNT': 'sale_price',
    'NAME': 'buyer_name'
})

acris.to_csv('acris_sales_final.csv', index=False)
print("\n✓ Saved final ACRIS sales data to acris_sales_final.csv")

# Summary statistics
print("\n=== FINAL SUMMARY ===")
print(f"Total property sales (2015+): {len(acris):,}")
print(f"Institutional purchases: {acris['is_institutional'].sum():,}")
print(f"Median sale price: ${acris['sale_price'].median():,.0f}")
print(f"Properties with BBL: {acris['BBL'].notna().sum():,}")


Merging all ACRIS data...


NameError: name 'master_clean' is not defined